In [1]:
from typing import List, Any, Dict
import sys

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Doc, Token
from allennlp.data.dataset_readers.dataset_utils.span_utils import bio_tags_to_spans
from allennlp.predictors import Predictor

sys.path.append('..')
import defx  # register custom classes

In [2]:
train_file_path = '/home/marc/code/dfki/semeval_2020_task_6/data/deft_split/jsonl/train.jsonl'

In [3]:
model_file_path = '/home/marc/research/semeval_2020_task_6/submission-candidates/2020-02-19__16-32__subtask2__single_char_crf_bert_with_pos/model.tar.gz'
predictor = Predictor.from_path(model_file_path, 'subtask1_predictor')

In [4]:
instances = predictor._dataset_reader.read(train_file_path)

6261it [00:02, 2495.56it/s]


In [5]:
list(instances[0].keys())

['metadata', 'tokens', 'tags']

In [6]:
" ".join([t.text for t in instances[0]['tokens']])

'5 . Science includes such diverse fields as astronomy , biology , computer sciences , geology , logic , physics , chemistry , and mathematics ( [ link ] ) . However , those fields of science related to the physical world and its phenomena and processes are considered natural sciences . Thus , a museum of natural sciences might contain any of the items listed above .'

In [7]:
def tokens_to_text(tokens):
    text = ''
    char_offsets = []
    for token in tokens:
        start_offset = len(text)
        text += token
        end_offset = len(text)
        char_offsets.append((start_offset, end_offset))
        text += ' '
    return text, char_offsets

In [8]:
def spans_to_spacy(spans, char_offsets):
    displacy_ents = []
    for span in spans:
        char_start = char_offsets[span[1][0]][0]
        char_end = char_offsets[span[1][1]][1]
        displacy_ents.append({
            'label': span[0],
            'start': char_start,
            'end': char_end
        })
    return displacy_ents

def displacy_example(tokens, tags):
    text, char_offsets = tokens_to_text(tokens)
    spans = bio_tags_to_spans(tags)
    spans = sorted(spans, key=lambda x: x[1][0])
    displacy_ents = spans_to_spacy(spans, char_offsets)
    displacy_output = {
        'text': text,
        'ents': displacy_ents,
        'title': None
    }
    colors = {
        'TERM': 'lightgreen',
        'DEFINITION': 'lightblue',
        'REFERENTIAL-TERM': '#34962d',
        'REFERENTIAL-DEFINITION': '#3348b0',
    }
    displacy.render(displacy_output, style='ent', manual=True, options={"colors": colors})
    
def displacy_instance(instance):
    tokens = [t.text for t in instance.fields['tokens']]
    tags = instance.fields['tags']
    displacy_example(tokens, tags)

In [9]:
displacy_instance(instances[0])

In [10]:
pipe = spacy.load('en_core_web_lg')

In [11]:
def instance_to_doc(instance):
    tokens = [t.text for t in instance.fields['tokens']]
    spaces = [True] * len(tokens)
    spaces[-1] = False
    doc = Doc(pipe.vocab, words=tokens, spaces=spaces)
    proc_doc = pipe.tagger(doc)
    return proc_doc
#def instance_to_doc(instance):
#    text = " ".join([t.text for t in instance.fields['tokens']])
#    return pipe(text)

In [12]:
#doc = instance_to_doc(instances[0])
docs = [instance_to_doc(i) for i in instances]

In [13]:
def print_matches(matches, doc):
    for match in matches:
        span = doc[match[1]:match[2]]
        print(f"({match[1]},{match[2]}): {span}")

In [22]:
# In between sentence starts: 'This is called' or 'These are called'
matcher = Matcher(pipe.vocab, validate=True)

ref_def_patterns = [
    [{'IS_DIGIT': False}, {'LOWER': '.'}, {'LOWER': 'this'}, {'OP': '+'}, {'LOWER': 'is'}, {'LOWER': 'called'}],
    [{'IS_DIGIT': False}, {'LOWER': '.'}, {'LOWER': 'these'}, {'OP': '+'}, {'LOWER': 'are'}, {'LOWER': 'called'}],
]

# Register Token extension
extensions = ['is_reference', 'is_definition', 'is_alias', 'noisy_alias', 'reference_keywords']
for extension in extensions:
    Token.set_extension(extension, default=False, force=True)
    
# Add a generic attribute setter
def extension_labeling(extension_name):
    def set_extension(matcher, doc, i, matches):
        match_id, start, end = matches[i]
        for token in doc[start:end]:
            token._.set(extension_name, True)
    return set_extension
        
matcher.add('is_reference', extension_labeling('is_reference'), *ref_def_patterns)
        
is_definition_patterns = [
    [{'POS': {'IN': ['NOUN', 'ADJ', 'DET']}, 'OP': '+'}, {'LOWER': 'is'}, {'LOWER': {'IN': ['a', 'the']}}]
]
matcher.add('is_definition', extension_labeling('is_definition'), *is_definition_patterns)

is_alias_patterns = [
    [{'LOWER': {'IN': [',', '(']}, 'OP': '?'}, {'LOWER': {'IN': ['formerly', 'commonly', 'also']}}, {'LOWER': 'called'}],
    [{'LOWER': {'IN': [',', '(']}, 'OP': '?'}, {'LOWER': {'IN': ['formerly', 'commonly', 'also']}}, {'LOWER': 'known'}, {'LOWER': 'as'}]
]
matcher.add('is_alias', extension_labeling('is_alias'), *is_alias_patterns)

noisy_alias_patterns = [
    [{'LOWER': '('}, {'LOWER': 'or', 'OP': '?'}, {'POS': {'IN': ['DET', 'ADJ', 'NOUN']}}, {'LOWER': ')'}],
]
matcher.add('noisy_alias', extension_labeling('noisy_alias'), *noisy_alias_patterns)

reference_keyword_patterns = [
    [{'LOWER': '.'}, {'LOWER': {'IN': ['it', 'they', 'these', 'those']}}]
]
matcher.add('reference_keywords', extension_labeling('reference_keywords'), *reference_keyword_patterns)

In [24]:
matched_docs = []
inst_and_docs = zip(instances, docs)

for instance, doc in inst_and_docs:
    if False:
        tags = instance.fields['tags']
        b_alias = 'B-Qualifier'
        i_alias = 'I-Qualifier'
        if b_alias in tags or i_alias in tags:
            matched_docs.append({
                'id': instance.fields['metadata']['example_id'],
                'matches': [],
                'doc': doc,
                'instance': instance
            })
    else:
        matches = matcher(doc)
        if len(matches) > 0:
            matched_docs.append({
                'id': instance.fields['metadata']['example_id'],
                'matches': matches,
                'doc': doc,
                'instance': instance
            })
len(matched_docs)

1704

In [21]:
for matched_doc in matched_docs[:100]:
    print('*' * 100)
    doc = matched_doc['doc']
    print_matches(matched_doc['matches'], doc)
    print('*' * 100)
    displacy_instance(matched_doc['instance'])
    print('*' * 100)

****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


...

****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


****************************************************************************************************


In [506]:
matched_doc = matched_docs[1]
doc = matched_doc['doc']
matches = matched_doc['matches']

In [507]:
matches

[(3443139886701419530, 7, 10), (3443139886701419530, 8, 10)]

In [508]:
doc

209 . Facilitated transport In facilitated transport , also called facilitated diffusion , materials diffuse across the plasma membrane with the help of membrane proteins . A concentration gradient exists that would allow these materials to diffuse into the cell without expending cellular energy .

In [511]:
for match in matches:
    for idx in range(match[1]-1, match[2]+1):
        print(idx, doc[idx], ' - ', doc[idx]._.get('is_alias'))
    print('*' * 60)

6 transport  -  False
7 ,  -  True
8 also  -  True
9 called  -  True
10 facilitated  -  False
************************************************************
7 ,  -  True
8 also  -  True
9 called  -  True
10 facilitated  -  False
************************************************************


In [452]:
doc[23].pos_

'DET'

In [468]:
[t for t in instances[0].fields['tags']][:10]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [25]:
len(instances)

6261